<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-GSC-Functions" data-toc-modified-id="Import-GSC-Functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import GSC Functions</a></span><ul class="toc-item"><li><span><a href="#Set-dates-&amp;-filenames" data-toc-modified-id="Set-dates-&amp;-filenames-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Set dates &amp; filenames</a></span></li><li><span><a href="#Check-verified-sites" data-toc-modified-id="Check-verified-sites-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Check verified sites</a></span></li></ul></li><li><span><a href="#Get-last-14-days-data" data-toc-modified-id="Get-last-14-days-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Get last 14 days data</a></span><ul class="toc-item"><li><span><a href="#Categorize-queries-in-brand,-cameras-and-generic" data-toc-modified-id="Categorize-queries-in-brand,-cameras-and-generic-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Categorize queries in brand, cameras and generic</a></span></li><li><span><a href="#Parse-dates" data-toc-modified-id="Parse-dates-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Parse dates</a></span></li></ul></li><li><span><a href="#Save-file" data-toc-modified-id="Save-file-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Save file</a></span><ul class="toc-item"><li><span><a href="#Save-csv-local" data-toc-modified-id="Save-csv-local-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Save csv local</a></span></li><li><span><a href="#Load-to-Big-Query" data-toc-modified-id="Load-to-Big-Query-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Load to Big Query</a></span></li><li><span><a href="#Load-to-Cloud-Storage" data-toc-modified-id="Load-to-Cloud-Storage-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Load to Cloud Storage</a></span></li></ul></li></ul></div>

In [1]:
from datetime import timedelta, datetime
from googleapiclient.discovery import build
from google.cloud import bigquery
from google.cloud import storage
from google.oauth2 import service_account
import calendar
import json
import pandas as pd
import re
import requests

pd.set_option('max_colwidth', 150)

# Import GSC Functions

In [2]:
%run 'GSC-Functions.ipynb'

## Set dates & filenames

In [3]:
first_available

datetime.datetime(2021, 1, 2, 11, 35, 21, 124613)

In [4]:
first_date = (today - timedelta(days=15)).strftime('%Y-%m-%d')
last_date = (today - timedelta(days=1)).strftime('%Y-%m-%d')

first_date, last_date

('2022-04-12', '2022-04-26')

In [5]:
#Get date range to be used on output file names

date_range = (first_date + '_' + last_date).replace("-","")
filename = ('gsc_daily_' + date_range + '.csv') 
localpath = ('files/' + filename)
filename, localpath

('gsc_daily_20220412_20220426.csv', 'files/gsc_daily_20220412_20220426.csv')

## Check verified sites

In [6]:
verified_sites_urls

['https://www.verisure.no/',
 'https://www.verisure.dk/',
 'https://www.verisure.it/',
 'https://www.verisure.cl/',
 'https://www.verisure.de/',
 'https://www.verisure.pe/',
 'https://www.verisure.fi/',
 'https://www.verisure.co.uk/',
 'https://www.securitasdirect.es/',
 'https://www.verisure.com.br/',
 'https://www.verisure.nl/',
 'https://www.verisure.com.ar/',
 'https://www.verisure.fr/',
 'https://www.verisure.se/',
 'https://www.verisure.be/',
 'https://www.securitasdirect.pt/']

# Get last 14 days data

In [7]:
#------------------------------------------- last 7 days data on countries loop ------------------------------------------

service = connect(key)
maxRows = 25000
i = 0
payload = {
    'startDate': first_date,
    'endDate': last_date,
    'dimensions': ['date', 'query', 'country', 'device'],
    'rowLimit': maxRows,
    'startRow': 0}

results = []

for s in verified_sites_urls:
    print(s)
    df_s = paginated(service, s, payload)
    payload['startRow'] = 0
    results.append(df_s)
    print('\n')
    
df = pd.concat(results)
df

https://www.verisure.no/
Working on loop 0
Working on loop 1
Working on loop 2


https://www.verisure.dk/
Working on loop 0
Working on loop 1
Working on loop 2


https://www.verisure.it/
Working on loop 0
Working on loop 1
Working on loop 2


https://www.verisure.cl/
Working on loop 0


https://www.verisure.de/
Working on loop 0


https://www.verisure.pe/
Working on loop 0


https://www.verisure.fi/
Working on loop 0


https://www.verisure.co.uk/
Working on loop 0
Working on loop 1
Working on loop 2
Working on loop 3
Working on loop 4
Working on loop 5


https://www.securitasdirect.es/
Working on loop 0
Working on loop 1
Working on loop 2
Working on loop 3
Working on loop 4


https://www.verisure.com.br/
Working on loop 0


https://www.verisure.nl/
Working on loop 0
Working on loop 1


https://www.verisure.com.ar/
Working on loop 0
Working on loop 1


https://www.verisure.fr/
Working on loop 0
Working on loop 1
Working on loop 2
Working on loop 3
Working on loop 4
Working on loop 5
Wor

,date,query,country,device,clicks,impressions,ctr,position,site_country
0,2022-04-19,verisure,nor,DESKTOP,63,240,26.25,1.13,no/
1,2022-04-21,verisure,nor,DESKTOP,62,202,30.69,1.16,no/
2,2022-04-20,verisure,nor,DESKTOP,60,204,29.41,1.21,no/
3,2022-04-19,verisure min side,nor,DESKTOP,54,64,84.38,1.00,no/
4,2022-04-20,verisure,nor,MOBILE,53,291,18.21,1.12,no/
...,...,...,...,...,...,...,...,...,...
17781,2022-04-24,www.securitasdirect.pt,prt,DESKTOP,0,1,0.00,1.00,pt/
17782,2022-04-24,zero vision,bra,DESKTOP,0,1,0.00,6.00,pt/
17783,2022-04-24,zerovision,gbr,DESKTOP,0,1,0.00,10.00,pt/
17784,2022-04-24,zerovision,prt,MOBILE,0,1,0.00,5.00,pt/


## Categorize queries in brand, cameras and generic

In [8]:
df = query_cats(df)

## Parse dates

In [9]:
df = date_datetime(df)

# Save file

## Save csv local

In [10]:
df.to_csv(localpath, index=False)

## Load to Big Query

In [ ]:
#------------------------------------------- solve credentials & env ------------------------------------------

location = 'credentials/client_secrets.json'

%env GOOGLE_APPLICATION_CREDENTIALS=$location

%load_ext google.cloud.bigquery

In [ ]:
table_schema = [{'name': "date", 'type': "DATE"},
                {'name': "query", 'type': "STRING"},
                {'name': "country", 'type': "STRING"},
                {'name': "device", 'type': "STRING"},
                {'name': "clicks", 'type': "INTEGER"},
                {'name': "impressions", 'type': "INTEGER"},
                {'name': "ctr", 'type': "FLOAT"},
                {'name': "position", 'type': "FLOAT"},
                {'name': "site_country", 'type': "STRING"},
                {'name': "query_cat", 'type': "STRING"}]

In [ ]:
#------------------------------------------- write on bq ------------------------------------------
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_gbq.html 

df.to_gbq(
    'SEO.gsc_daily_test', #dataset.table
    'group-lead-gen', #project
    chunksize=1000000,
    table_schema=table_schema, #optional
    if_exists='append'
)

## Load to Cloud Storage

In [11]:
#------------------------------------------- solve credentials & env ------------------------------------------

location = 'credentials/client_secrets.json'

%env GOOGLE_APPLICATION_CREDENTIALS =$location

storage_client = storage.Client()

env: GOOGLE_APPLICATION_CREDENTIALS=credentials/client_secrets.json


In [13]:
bucket = storage_client.get_bucket('gsc_daily')

def save_to_google_cloud_storage(destination_file, source_data, is_string_data):
    blob = bucket.blob(destination_file)
    if is_string_data:
        blob.upload_from_string(source_data)
    else:
        blob.upload_from_filename(source_data)

In [14]:
save_to_google_cloud_storage(filename, df.to_csv(index=False), True)